In [ ]:
# initial imports:
import sys, os
os.environ['TF_USE_LEGACY_KERAS'] = '1'  # needed for tensorflow KERAS compatibility
os.environ['DISPLAY'] = 'inline'  # hack to get getdist working
import matplotlib.pyplot as plt
import numpy as np

# tensiometer local imports:
here = os.path.dirname(os.getcwd())
tensiometer_path = os.path.join(here, 'tensiometer')
sys.path.append(tensiometer_path)

import tensiometer
print('Tensiometer version:', tensiometer.__version__)
print('Tensiometer path:', tensiometer.__path__)

from cobaya import load_samples
from cobaya.model import get_model
import yaml
from getdist import plots

## Load the chains

In [ ]:
# chains paths:
chains_dir = os.path.join('chains')

# getdist settings:
getdist_settings = {
    'smooth_scale_1D': 0.3,
    'boundary_correction_order': 1,
    'mult_bias_correction_order': 1,
    'smooth_scale_2D': 0.3,
    }

### Planck

In [ ]:
planck = '01p2_w0wa_CMB_PlanckTTTEEE'
chain_planck = load_samples(os.path.join(chains_dir, planck), skip=0.3, to_getdist=True)
chain_planck.label = 'Planck TTTEEE'
chain_planck.updateSettings(getdist_settings)
chain_planck.updateBaseStatistics()
chain_planck.getConvergeTests()
print(chain_planck.label+': R =', round(chain_planck.GelmanRubin, 4))
#chain_planck.getParamNames().list()

### ACT

In [ ]:
act = '01p3_w0wa_CMB_ACTTTTEEE'
chain_act = load_samples(os.path.join(chains_dir, act), skip=0.3, to_getdist=True)
chain_act.label = 'ACT TTTEEE'
chain_act.updateSettings(getdist_settings)
chain_act.updateBaseStatistics()
chain_act.getConvergeTests()
print(chain_act.label+': R =', round(chain_act.GelmanRubin, 4))
#chain_act.getParamNames().list()

### CMB $_{all}$

In [ ]:
cmb_all = '02p2_w0wa_CMB_all'
chain_cmb_all = load_samples(os.path.join(chains_dir, cmb_all), skip=0.3, to_getdist=True)
chain_cmb_all.label = 'Planck TTTEEE + ACT TTTEEE + lowl'
chain_cmb_all.updateSettings(getdist_settings)
chain_cmb_all.updateBaseStatistics()
chain_cmb_all.getConvergeTests()
print(chain_cmb_all.label+': R =', round(chain_cmb_all.GelmanRubin, 4))
#chain_cmb_all.getParamNames().list()

### CMB + BAO + SN

In [ ]:
cmb_bao_sn = '04p1_w0wa_CMB_BAO_SN'
chain_cmb_bao_sn = load_samples(os.path.join(chains_dir, cmb_bao_sn), skip=0.3, to_getdist=True)
chain_cmb_bao_sn.label = 'CMB + BAO + SN'
chain_cmb_bao_sn.updateSettings(getdist_settings)
chain_cmb_bao_sn.updateBaseStatistics()
chain_cmb_bao_sn.getConvergeTests()
print(chain_cmb_bao_sn.label+': R =', round(chain_cmb_bao_sn.GelmanRubin, 4))
#chain_cmb_bao_sn.getParamNames().list()

**Rebuild the model**

In [ ]:
names = [planck, act, cmb_all, cmb_bao_sn]

# define the path to the YAML parameters file:
for name in names:
    yaml_params = os.path.join(chains_dir, name + '.input.yaml')
    # load the parameters from the YAML file:
    with open(yaml_params, 'r') as f:
        params_dict = yaml.safe_load(f)
    # remove the 'path' key from the 'camb' dictionary if it exists:
    if 'path' in params_dict.get('theory', {}).get('camb', {}):
        del params_dict['theory']['camb']['path']
    if 'recombination_model' in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
        del params_dict['theory']['camb']['extra_args']['recombination_model']

    # build the model:
    model = get_model(params_dict)

## Triangle plot

In [ ]:
params = ['H0', 'ombh2', 'omch2', 'As', 'w', 'wa']
g = plots.getSubplotPlotter()
g.triangle_plot([chain_planck, chain_act, chain_cmb_all, chain_cmb_bao_sn],
                params=params,
                filled=True, legend=True, legend_loc='upper right',
                colors=['gold', 'limegreen', 'deepskyblue', 'mediumorchid'],
                contour_colors=['gold', 'limegreen', 'deepskyblue', 'mediumorchid'],
)

**Covariance**

In [ ]:
cov = np.loadtxt("chains/01p2_w0wa_CMB_PlanckTTTEEE.covmat")
print(cov)
dim = len(cov)
print(dim)